In [1]:
#we load our libraries
import pandas as pd
import sys
sys.path.append('/Users/halukamb/Documents/CodeRelated/HBDataviz_new/2025/01_BTW100Deutsche/Wrangle/lib/python3.12/site-packages')
import numpy as np
from json import loads, dumps

In [2]:
! pip install pyreadstat


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
#we have one function that filters the ALLBUS data for whether respondents are german and of voting age
def germanElectorate(df):
    german_df=df[df["german"]=="JA"]
    german_df=german_df.dropna(subset="age")
    german_df["age"]=german_df["age"].astype(float)
    german_electorate=german_df[german_df["age"]>=18]
    return german_electorate

In [4]:
#we have another function that regroups the data into fitting categories to the data we have from the zensus
def reformgroups(df):
    df["Alter"]=None
    df["Alter"]=np.where(df["age"]<30,"U30",df["Alter"])
    df["Alter"]=np.where(df["age"]>=30,"30-49",df["Alter"])
    df["Alter"]=np.where(df["age"]>=50,"50-64",df["Alter"])
    df["Alter"]=np.where(df["age"]>64,"Ü65",df["Alter"])
    df["Geschlecht"]=np.nan
    df["Geschlecht"]=np.where(df["sex"]=="FRAU","Weiblich",df["Geschlecht"])
    df["Geschlecht"]=np.where(df["sex"]=="MANN","Männlich",df["Geschlecht"])
    df["OstWest"]=np.nan
    df["OstWest"]=np.where(df["eastwest"]=="ALTE BUNDESLAENDER","West",df["OstWest"])
    df["OstWest"]=np.where(df["eastwest"]=="NEUE BUNDESLAENDER","Ost",df["OstWest"])
    df["OstWest"]=np.where(df["land"].str.contains("BERLIN"),"Berlin",df["OstWest"])

    df["Besiedelung"]=np.nan
    df["Besiedelung"]=np.where(df["gs01"].isin(['GROSSSTADT']),"dicht besiedelt",df["Besiedelung"])
    df["Besiedelung"]=np.where(df["gs01"].isin( ['MITTEL-, KLEINSTADT', 'VORORT GROSSSTADT']),"mittlere Besiedlungsdichte",df["Besiedelung"])
    df["Besiedelung"]=np.where(df["gs01"].isin( ['LAENDL. DORF','EINZELHAUS, LAND']),"gering besiedelt",df["Besiedelung"])
    
    


    
    return df

In [5]:
#and lastly we have a stratification function that takes the zensus data and the skewed dataset of allbus
#  and creates a stratified dataset based on the population shares of the zensus data
#for each combination of age, gender, eastwest and population density
def stratifier(zensus_model,skewed_df):

    stratified_dataset=pd.DataFrame()
    samplesize_sum=0

    #we go through each row of the zensus groups
    for index, row in zensus_model.iterrows():
        #take in which share of the population this group represents
        samplesize = round(row["Share"] * 100)
        samplesize_sum += samplesize
        #and then we take a copy of the skewed allbus data
        sub_df = skewed_df.copy()
        
        for i in range(0, 4):
            #and then we filter it down, feature by feature of the speficic row of the zensus data
            #we do this from 0 to 4 because in the first four rows we have the features we want to stratify by
            sub_df = sub_df[sub_df[row.index[i]] == row.values[i]]
            print(row.index[i],row.values[i],len(sub_df))
        
        
        #we draw a sample from this group that correlates to the share of the population
        #if the original dataset is too small, we sample with replacement
        sampled_subgroup = sub_df.sample(samplesize, random_state=1, replace=True)
        #and add this sampled group to the overall stratified dataset
        stratified_dataset = pd.concat([stratified_dataset, sampled_subgroup])
    print(len(stratified_dataset),samplesize_sum)
    return stratified_dataset


In [6]:
#this is the zensus data we need, we remove all rows with NaN values
#as we need as detailed data as possible for the stratification
zensus_model=pd.read_csv("zensus_model.csv")
zensus_model=zensus_model.dropna()
zensus_model=zensus_model.sort_values(by="Share",ascending=False)


In [7]:
#we read in the allbus data for 2021
df_2021 = pd.read_spss("../data/AllbusData/ZA5280_v2-0-1.sav")
# filter it for the german electorate

df_2021 = germanElectorate(df_2021)
# regroup it to fit the zensus data

df_2021= reformgroups(df_2021)
# and stratify it based on the zensus data
df_2021= stratifier(zensus_model,df_2021)
df_2021

Geschlecht Weiblich 2474
Besiedelung mittlere Besiedlungsdichte 1204
OstWest West 744
Alter Ü65 223
Geschlecht Weiblich 2474
Besiedelung mittlere Besiedlungsdichte 1204
OstWest West 744
Alter 50-64 247
Geschlecht Männlich 2419
Besiedelung mittlere Besiedlungsdichte 1182
OstWest West 752
Alter 50-64 233
Geschlecht Weiblich 2474
Besiedelung mittlere Besiedlungsdichte 1204
OstWest West 744
Alter 30-49 193
Geschlecht Männlich 2419
Besiedelung mittlere Besiedlungsdichte 1182
OstWest West 752
Alter 30-49 176
Geschlecht Männlich 2419
Besiedelung mittlere Besiedlungsdichte 1182
OstWest West 752
Alter Ü65 263
Geschlecht Weiblich 2474
Besiedelung dicht besiedelt 465
OstWest West 250
Alter Ü65 60
Geschlecht Männlich 2419
Besiedelung dicht besiedelt 451
OstWest West 247
Alter 30-49 79
Geschlecht Weiblich 2474
Besiedelung dicht besiedelt 465
OstWest West 250
Alter 30-49 83
Geschlecht Weiblich 2474
Besiedelung dicht besiedelt 465
OstWest West 250
Alter 50-64 60
Geschlecht Männlich 2419
Besiedelung d

,za_nr,doi,version,respid,substudy,mode,splt21,eastwest,german,ep01,...,bik,gkpol,wghtpew,wghtht,wghthew,wghthtew,Alter,Geschlecht,OstWest,Besiedelung
986,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),987.0,SIMULTAN,MAIL,SPLIT A,ALTE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 1 ;>499.999,100.000 - 499.999 E.,ALTE BUNDESLAENDER,1.685751,ALTE BUNDESLAENDER,2.083416,Ü65,Weiblich,West,mittlere Besiedlungsdichte
3459,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),3460.0,SIMULTAN,MAIL,SPLIT C,ALTE BUNDESLAENDER,JA,NaN,...,ZONE 1 ;<500.000,100.000 - 499.999 E.,ALTE BUNDESLAENDER,NaN,NaN,NaN,Ü65,Weiblich,West,mittlere Besiedlungsdichte
1790,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),1791.0,SIMULTAN,MAIL,SPLIT A,ALTE BUNDESLAENDER,JA,GUT,...,ZONE 2-4;>499.999,20.000 - 49.999 EINW,ALTE BUNDESLAENDER,0.842875,ALTE BUNDESLAENDER,1.041708,Ü65,Weiblich,West,mittlere Besiedlungsdichte
3381,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),3382.0,SIMULTAN,MAIL,SPLIT A,ALTE BUNDESLAENDER,JA,NaN,...,ZONE 1 ;>499.999,500.000 UND MEHR E.,ALTE BUNDESLAENDER,1.685751,ALTE BUNDESLAENDER,2.083416,Ü65,Weiblich,West,mittlere Besiedlungsdichte
4800,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4801.0,SEQUENZIELL,CAWI,SPLIT C,ALTE BUNDESLAENDER,JA,NaN,...,ZONE 2-4;<500.000,5.000 - 19.999 EINW,ALTE BUNDESLAENDER,0.842875,ALTE BUNDESLAENDER,1.041708,Ü65,Weiblich,West,mittlere Besiedlungsdichte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4710,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4711.0,SIMULTAN,MAIL,SPLIT A,NEUE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 2-4;<100.000,2.000 - 4.999 EINW,NEUE BUNDESLAENDER,0.267321,NEUE BUNDESLAENDER,0.141565,U30,Weiblich,Ost,gering besiedelt
1464,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),1465.0,SIMULTAN,MAIL,SPLIT B,NEUE BUNDESLAENDER,JA,SCHLECHT,...,ZONE 2-4;<500.000,20.000 - 49.999 EINW,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695,U30,Weiblich,Ost,gering besiedelt
4710,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4711.0,SIMULTAN,MAIL,SPLIT A,NEUE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 2-4;<100.000,2.000 - 4.999 EINW,NEUE BUNDESLAENDER,0.267321,NEUE BUNDESLAENDER,0.141565,U30,Weiblich,Ost,gering besiedelt
2291,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),2292.0,SIMULTAN,MAIL,SPLIT A,NEUE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 2-4;>499.999,5.000 - 19.999 EINW,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695,U30,Weiblich,Ost,gering besiedelt


In [47]:
#we read in the allbus data for 2023
df_2023 = pd.read_spss("../data/AllbusData/ZA8830_v1-0-0.sav")
# filter it for the german electorate

df_2023 = germanElectorate(df_2023)
# regroup it to fit the zensus data

df_2023= reformgroups(df_2023)
#and straitify it based on the zensus data
df_2023= stratifier(zensus_model,df_2023)
df_2023


Geschlecht Weiblich 2412
Besiedelung mittlere Besiedlungsdichte 1224
OstWest West 729
Alter Ü65 208
Geschlecht Weiblich 2412
Besiedelung mittlere Besiedlungsdichte 1224
OstWest West 729
Alter 50-64 230
Geschlecht Männlich 2292
Besiedelung mittlere Besiedlungsdichte 1078
OstWest West 666
Alter 50-64 184
Geschlecht Weiblich 2412
Besiedelung mittlere Besiedlungsdichte 1224
OstWest West 729
Alter 30-49 197
Geschlecht Männlich 2292
Besiedelung mittlere Besiedlungsdichte 1078
OstWest West 666
Alter 30-49 178
Geschlecht Männlich 2292
Besiedelung mittlere Besiedlungsdichte 1078
OstWest West 666
Alter Ü65 222
Geschlecht Weiblich 2412
Besiedelung dicht besiedelt 485
OstWest West 259
Alter Ü65 76
Geschlecht Männlich 2292
Besiedelung dicht besiedelt 471
OstWest West 257
Alter 30-49 85
Geschlecht Weiblich 2412
Besiedelung dicht besiedelt 485
OstWest West 259
Alter 30-49 60
Geschlecht Weiblich 2412
Besiedelung dicht besiedelt 485
OstWest West 259
Alter 50-64 64
Geschlecht Männlich 2292
Besiedelung d

,za_nr,doi,version,respid,substudy,mode,splt23_1,splt23_2,eastwest,german,...,bik,xs11,wghtpew,wghtht,wghthew,wghthtew,Alter,Geschlecht,OstWest,Besiedelung
770,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",771.0,NaN,CAPI,NaN,SPLIT A,ALTE BUNDESLAENDER,JA,...,ZONE 2-4;>499.999,183.0,1.252326,1.69639,1.242308,2.107439,Ü65,Weiblich,West,mittlere Besiedlungsdichte
3538,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",3539.0,NaN,CAPI,NaN,SPLIT B,ALTE BUNDESLAENDER,JA,...,ZONE 1 ;>499.999,93.0,1.252326,0.848195,1.242308,1.053719,Ü65,Weiblich,West,mittlere Besiedlungsdichte
1776,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",1777.0,SIMULTAN,MAIL,SPLIT B,TNZ: MODE,ALTE BUNDESLAENDER,JA,...,ZONE 1-4;<50.000,10.0,1.203727,1.693259,1.192166,2.018646,Ü65,Weiblich,West,mittlere Besiedlungsdichte
3432,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",3433.0,SEQUENZIELL,CAWI,SPLIT A,TNZ: MODE,ALTE BUNDESLAENDER,JA,...,ZONE 1 ;<500.000,168.0,1.207467,0.554394,1.205184,0.668147,Ü65,Weiblich,West,mittlere Besiedlungsdichte
5150,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",5151.0,NaN,CAPI,NaN,SPLIT B,ALTE BUNDESLAENDER,JA,...,ZONE 2-4;>499.999,143.0,1.252326,0.848195,1.242308,1.053719,Ü65,Weiblich,West,mittlere Besiedlungsdichte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1897,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",1898.0,NaN,CAPI,NaN,SPLIT B,NEUE BUNDESLAENDER,JA,...,5.000-19.999 EINW.,55.0,0.501346,0.532744,0.524886,0.27963,U30,Weiblich,Ost,gering besiedelt
3272,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",3273.0,SIMULTAN,MAIL,SPLIT A,TNZ: MODE,NEUE BUNDESLAENDER,JA,...,ZONE 1 ;<500.000,176.0,0.544816,0.845091,0.572064,0.483446,U30,Weiblich,Ost,gering besiedelt
4876,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",4877.0,SIMULTAN,MAIL,SPLIT A,TNZ: MODE,NEUE BUNDESLAENDER,JA,...,ZONE 2-4;<500.000,31.0,0.544816,0.338036,0.572064,0.193378,U30,Weiblich,Ost,gering besiedelt
4876,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",4877.0,SIMULTAN,MAIL,SPLIT A,TNZ: MODE,NEUE BUNDESLAENDER,JA,...,ZONE 2-4;<500.000,31.0,0.544816,0.338036,0.572064,0.193378,U30,Weiblich,Ost,gering besiedelt


In [48]:
df_2021["ma01b"].value_counts()

ma01b
..                     4267
STIMME VOLL ZU         1831
STIMME GAR NICHT ZU     280
Name: count, dtype: int64

In [49]:
df_2023["ma01b"].value_counts()

ma01b
..                     4706
STIMME VOLL ZU         1948
STIMME GAR NICHT ZU     215
Name: count, dtype: int64

In [9]:
#this is a function that enables us to analyze for each for each subgroup how the respondents answered differently
#  to a given question/variable
def analyze_factor_patterns(dataset,variable_to_check):
    #we have a base dataframe that holds all results
    base_df=pd.DataFrame()
    #now we define the four features we want to use to check for changes in the respondents data in allbus
    factor_list=["Alter","Geschlecht","OstWest","Besiedelung"]
    #we iterate through all possible combinations of these features
    for element in factor_list:
        #print(element)
        for element2 in factor_list:
            #add we za_nr to the list of the variable to check, as it is the unique identifier for each respondent
            baselist=[variable_to_check,"za_nr"]

            #we build also a list of the features we want to check for
            extralist=[element,element2]
            #set it to remove duplicates, in case we pick the same feature twice
            extralist=list(set(extralist))

            #and combine the two lists and filter the dataset for the columns we need
            baselist.extend(extralist)
            dataset_reduced=dataset[baselist]
            #remove all rows with missing values
            dataset_reduced = dataset_reduced[~dataset_reduced.isin(["nan"]).any(axis=1)]

            #we also define a extralist plus which we need to group by the dataset
            #it is the name of the features we want to check for plus the variable
            # and then we count
            extralist_plus_factor=extralist+[variable_to_check]
            grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
            #we remove all columns that have missing values
            grouped_df=grouped_df.dropna(axis=1)
            #and then we calculate the share of the respondents for each subgroup by using the sum of the group
            grouped_df['percent'] = grouped_df['za_nr']*100 / grouped_df.groupby(by=extralist)['za_nr'].transform('sum')
            grouped_df['absolute'] = grouped_df['za_nr']
            
            #and we add this to the base dataframe that holds all results
            grouped_df=grouped_df.reset_index()
            base_df=pd.concat([base_df,grouped_df],axis=0)
    #at the end we drop duplicates
    base_df=base_df.drop_duplicates()
    #reduce the dataset
    base_df_cols=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', variable_to_check,
       'absolute','percent']
    #and return the dataset
    return base_df[base_df_cols]

In [10]:
# now we do this whole procedure for variable va03 which is for inflation
factor_to_check="ma01b"
df_2023_factor=analyze_factor_patterns(df_2023,factor_to_check)
df_2023_factor=df_2023_factor.sort_values(by="absolute",ascending=False)
df_2023_factor

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_58206/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_58206/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_58206/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,ma01b,absolute,percent
6,NaN,NaN,NaN,West,..,3876,71.289314
3,NaN,NaN,Weiblich,NaN,..,2400,67.529544
0,NaN,NaN,Männlich,NaN,..,2306,69.562594
6,NaN,mittlere Besiedlungsdichte,NaN,NaN,..,2105,67.706658
15,NaN,NaN,Weiblich,West,..,1970,69.660537
...,...,...,...,...,...,...,...
3,NaN,gering besiedelt,NaN,Berlin,..,0,NaN
4,NaN,gering besiedelt,NaN,Berlin,STIMME GAR NICHT ZU,0,NaN
8,NaN,mittlere Besiedlungsdichte,NaN,Berlin,STIMME VOLL ZU,0,NaN
7,NaN,mittlere Besiedlungsdichte,NaN,Berlin,STIMME GAR NICHT ZU,0,NaN


In [11]:
# now we do this whole procedure for variable va03 which is for inflation

df_2021_factor=analyze_factor_patterns(df_2021,factor_to_check)
df_2021_factor=df_2021_factor.sort_values(by="absolute",ascending=False)
df_2021_factor

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_58206/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_58206/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_58206/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,ma01b,absolute,percent
6,NaN,NaN,NaN,West,..,3473,68.867737
3,NaN,NaN,Weiblich,NaN,..,2170,64.989518
0,NaN,NaN,Männlich,NaN,..,2097,69.002962
6,NaN,mittlere Besiedlungsdichte,NaN,NaN,..,1863,66.157670
15,NaN,NaN,Weiblich,West,..,1785,67.054846
...,...,...,...,...,...,...,...
7,NaN,mittlere Besiedlungsdichte,NaN,Berlin,STIMME GAR NICHT ZU,0,NaN
8,U30,NaN,NaN,Berlin,STIMME VOLL ZU,0,0.000000
8,NaN,mittlere Besiedlungsdichte,NaN,Berlin,STIMME VOLL ZU,0,NaN
5,NaN,gering besiedelt,NaN,Berlin,STIMME VOLL ZU,0,NaN


In [12]:
# and we merge the datasets of 2021 and 2023 based on all the features
comp_factor=pd.merge(df_2021_factor,df_2023_factor,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest',factor_to_check],suffixes=["_2021","_2023"])


In [13]:
comp_factor=comp_factor[comp_factor[factor_to_check]=='STIMME VOLL ZU']
comp_factor

,Alter,Besiedelung,Geschlecht,OstWest,ma01b,absolute_2021,percent_2021,absolute_2023,percent_2023
8,NaN,NaN,NaN,West,STIMME VOLL ZU,1350,26.769780,1403,25.804672
14,NaN,NaN,Weiblich,NaN,STIMME VOLL ZU,990,29.649596,1032,29.037704
18,Ü65,NaN,NaN,NaN,STIMME VOLL ZU,883,46.449237,853,41.793239
19,NaN,mittlere Besiedlungsdichte,NaN,NaN,STIMME VOLL ZU,845,30.007102,921,29.623673
20,NaN,NaN,Männlich,NaN,STIMME VOLL ZU,841,27.673577,916,27.631976
...,...,...,...,...,...,...,...,...,...
162,U30,dicht besiedelt,NaN,NaN,STIMME VOLL ZU,19,4.460094,28,6.436782
164,30-49,NaN,NaN,Berlin,STIMME VOLL ZU,18,18.556701,7,7.865169
191,U30,NaN,NaN,Berlin,STIMME VOLL ZU,0,0.000000,1,2.222222
192,NaN,mittlere Besiedlungsdichte,NaN,Berlin,STIMME VOLL ZU,0,NaN,0,NaN


In [14]:
comp_factor=comp_factor[comp_factor.columns[comp_factor.columns!=factor_to_check]]

In [15]:
non_berlin_comp=comp_factor[comp_factor["OstWest"]!="Berlin"]
non_berlin_comp

,Alter,Besiedelung,Geschlecht,OstWest,absolute_2021,percent_2021,absolute_2023,percent_2023
8,NaN,NaN,NaN,West,1350,26.769780,1403,25.804672
14,NaN,NaN,Weiblich,NaN,990,29.649596,1032,29.037704
18,Ü65,NaN,NaN,NaN,883,46.449237,853,41.793239
19,NaN,mittlere Besiedlungsdichte,NaN,NaN,845,30.007102,921,29.623673
20,NaN,NaN,Männlich,NaN,841,27.673577,916,27.631976
26,NaN,NaN,Weiblich,West,737,27.685950,773,27.333805
29,Ü65,NaN,NaN,West,660,44.265594,607,38.835573
30,NaN,mittlere Besiedlungsdichte,NaN,West,658,27.928693,702,27.052023
33,NaN,NaN,Männlich,West,613,25.745485,630,24.147183
40,Ü65,NaN,Weiblich,NaN,524,46.785714,521,45.984113


In [16]:
non_berlin_comp["diff"]=non_berlin_comp["percent_2023"]-non_berlin_comp["percent_2021"]
non_berlin_comp=non_berlin_comp.sort_values(by="diff",ascending=False)
non_berlin_comp

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_58206/3213792933.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_berlin_comp["diff"]=non_berlin_comp["percent_2023"]-non_berlin_comp["percent_2021"]


,Alter,Besiedelung,Geschlecht,OstWest,absolute_2021,percent_2021,absolute_2023,percent_2023,diff
104,50-64,NaN,NaN,Ost,109,35.620915,155,47.112462,11.491547
84,NaN,NaN,Männlich,Ost,193,37.044146,263,45.739130,8.694985
75,50-64,mittlere Besiedlungsdichte,NaN,NaN,229,28.236745,309,35.640138,7.403394
101,50-64,dicht besiedelt,NaN,NaN,115,20.034843,167,27.022654,6.987811
95,NaN,gering besiedelt,NaN,Ost,153,44.347826,191,51.206434,6.858608
68,50-64,NaN,Männlich,NaN,265,29.642058,336,35.257083,5.615025
44,50-64,NaN,NaN,NaN,484,27.114846,607,31.713689,4.598843
119,NaN,dicht besiedelt,NaN,Ost,74,28.906250,90,33.210332,4.304082
50,NaN,NaN,NaN,Ost,414,39.019793,500,43.177893,4.158100
60,50-64,NaN,NaN,West,350,24.857955,437,28.655738,3.797783


In [17]:
non_berlin_comp.head(30)

,Alter,Besiedelung,Geschlecht,OstWest,absolute_2021,percent_2021,absolute_2023,percent_2023,diff
104,50-64,NaN,NaN,Ost,109,35.620915,155,47.112462,11.491547
84,NaN,NaN,Männlich,Ost,193,37.044146,263,45.739130,8.694985
75,50-64,mittlere Besiedlungsdichte,NaN,NaN,229,28.236745,309,35.640138,7.403394
101,50-64,dicht besiedelt,NaN,NaN,115,20.034843,167,27.022654,6.987811
95,NaN,gering besiedelt,NaN,Ost,153,44.347826,191,51.206434,6.858608
68,50-64,NaN,Männlich,NaN,265,29.642058,336,35.257083,5.615025
44,50-64,NaN,NaN,NaN,484,27.114846,607,31.713689,4.598843
119,NaN,dicht besiedelt,NaN,Ost,74,28.906250,90,33.210332,4.304082
50,NaN,NaN,NaN,Ost,414,39.019793,500,43.177893,4.158100
60,50-64,NaN,NaN,West,350,24.857955,437,28.655738,3.797783


In [18]:
non_berlin_comp.tail(30)

,Alter,Besiedelung,Geschlecht,OstWest,absolute_2021,percent_2021,absolute_2023,percent_2023,diff
102,30-49,gering besiedelt,NaN,NaN,114,31.491713,117,31.451613,-0.040100
20,NaN,NaN,Männlich,NaN,841,27.673577,916,27.631976,-0.041601
59,NaN,dicht besiedelt,NaN,West,355,20.615563,366,20.412716,-0.202847
66,NaN,dicht besiedelt,Weiblich,NaN,272,22.723475,271,22.508306,-0.215170
83,Ü65,gering besiedelt,NaN,NaN,196,53.116531,231,52.860412,-0.256119
77,NaN,NaN,Weiblich,Ost,221,40.925926,237,40.651801,-0.274125
26,NaN,NaN,Weiblich,West,737,27.685950,773,27.333805,-0.352146
19,NaN,mittlere Besiedlungsdichte,NaN,NaN,845,30.007102,921,29.623673,-0.383429
42,NaN,gering besiedelt,NaN,NaN,490,37.404580,526,36.990155,-0.414425
41,NaN,dicht besiedelt,NaN,NaN,496,22.024867,501,21.428571,-0.596295


In [19]:
#also we do the same for party preferences
df_2021_party=analyze_factor_patterns(df_2021,"pv01")
df_2021_party

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_58206/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_58206/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_58206/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute,percent
0,30-49,NaN,NaN,NaN,AFD,137,6.135244
1,30-49,NaN,NaN,NaN,ANDERE PARTEI,114,5.105240
2,30-49,NaN,NaN,NaN,CDU-CSU,477,21.361397
3,30-49,NaN,NaN,NaN,DIE GRUENEN,743,33.273623
4,30-49,NaN,NaN,NaN,DIE LINKE,143,6.403941
...,...,...,...,...,...,...,...
19,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE GRUENEN,881,25.403691
20,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE LINKE,200,5.767013
21,NaN,mittlere Besiedlungsdichte,NaN,NaN,FDP,471,13.581315
22,NaN,mittlere Besiedlungsdichte,NaN,NaN,SPD,537,15.484429


In [20]:
df_2023_party=analyze_factor_patterns(df_2023,"pv01")
df_2023_party

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_58206/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_58206/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_58206/1110048911.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute,percent
0,30-49,NaN,NaN,NaN,AFD,271,12.212708
1,30-49,NaN,NaN,NaN,ANDERE PARTEI,77,3.470032
2,30-49,NaN,NaN,NaN,CDU-CSU,426,19.197837
3,30-49,NaN,NaN,NaN,DIE GRUENEN,534,24.064894
4,30-49,NaN,NaN,NaN,DIE LINKE,138,6.219018
...,...,...,...,...,...,...,...
19,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE GRUENEN,711,19.921547
20,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE LINKE,158,4.427010
21,NaN,mittlere Besiedlungsdichte,NaN,NaN,FDP,381,10.675259
22,NaN,mittlere Besiedlungsdichte,NaN,NaN,SPD,721,20.201737


In [21]:
df_2023_party[df_2023_party.percent.isna()]

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute,percent
8,NaN,gering besiedelt,NaN,Berlin,AFD,0,NaN
9,NaN,gering besiedelt,NaN,Berlin,ANDERE PARTEI,0,NaN
10,NaN,gering besiedelt,NaN,Berlin,CDU-CSU,0,NaN
11,NaN,gering besiedelt,NaN,Berlin,DIE GRUENEN,0,NaN
12,NaN,gering besiedelt,NaN,Berlin,DIE LINKE,0,NaN
13,NaN,gering besiedelt,NaN,Berlin,FDP,0,NaN
14,NaN,gering besiedelt,NaN,Berlin,SPD,0,NaN
15,NaN,gering besiedelt,NaN,Berlin,WUERDE NICHT WAEHLEN,0,NaN
16,NaN,mittlere Besiedlungsdichte,NaN,Berlin,AFD,0,NaN
17,NaN,mittlere Besiedlungsdichte,NaN,Berlin,ANDERE PARTEI,0,NaN


In [22]:
df_2021_party[df_2021_party.percent.isna()]

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute,percent
8,NaN,gering besiedelt,NaN,Berlin,AFD,0,NaN
9,NaN,gering besiedelt,NaN,Berlin,ANDERE PARTEI,0,NaN
10,NaN,gering besiedelt,NaN,Berlin,CDU-CSU,0,NaN
11,NaN,gering besiedelt,NaN,Berlin,DIE GRUENEN,0,NaN
12,NaN,gering besiedelt,NaN,Berlin,DIE LINKE,0,NaN
13,NaN,gering besiedelt,NaN,Berlin,FDP,0,NaN
14,NaN,gering besiedelt,NaN,Berlin,SPD,0,NaN
15,NaN,gering besiedelt,NaN,Berlin,WUERDE NICHT WAEHLEN,0,NaN
16,NaN,mittlere Besiedlungsdichte,NaN,Berlin,AFD,0,NaN
17,NaN,mittlere Besiedlungsdichte,NaN,Berlin,ANDERE PARTEI,0,NaN


In [23]:
#we also merge the datasets of 2021 and 2023 based on all the features for the party preferences
comp_party=pd.merge(df_2021_party,df_2023_party,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', 'pv01'],suffixes=["_2021","_2023"])
comp_party["diff_percent"]=comp_party["percent_2023"]-comp_party["percent_2021"]
comp_party.sort_values("diff_percent")

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute_2021,percent_2021,absolute_2023,percent_2023,diff_percent
110,50-64,NaN,NaN,Berlin,SPD,19,29.230769,7,9.459459,-19.771310
149,U30,NaN,NaN,Ost,FDP,28,22.580645,5,3.759398,-18.821247
119,U30,NaN,NaN,Berlin,WUERDE NICHT WAEHLEN,9,18.000000,0,0.000000,-18.000000
122,Ü65,NaN,NaN,Berlin,CDU-CSU,30,35.294118,17,18.279570,-17.014548
114,U30,NaN,NaN,Berlin,CDU-CSU,8,16.000000,0,0.000000,-16.000000
...,...,...,...,...,...,...,...,...,...,...
443,NaN,mittlere Besiedlungsdichte,NaN,Berlin,DIE GRUENEN,0,NaN,0,NaN,NaN
444,NaN,mittlere Besiedlungsdichte,NaN,Berlin,DIE LINKE,0,NaN,0,NaN,NaN
445,NaN,mittlere Besiedlungsdichte,NaN,Berlin,FDP,0,NaN,0,NaN,NaN
446,NaN,mittlere Besiedlungsdichte,NaN,Berlin,SPD,0,NaN,0,NaN,NaN


In [24]:

#and we pivot it based on the party preferences
comp_party_pivot=comp_party.pivot(index=['Alter','Besiedelung','Geschlecht','OstWest'], columns='pv01')[['percent_2021','percent_2023',"absolute_2021","absolute_2023"]].reset_index()
comp_party_pivot

Alter                 Besiedelung Geschlecht OstWest percent_2021  \
pv01                                                               AFD   
0      NaN                         NaN        NaN  Berlin     2.950820   
1      NaN                         NaN        NaN     Ost    11.746522   
2      NaN                         NaN        NaN    West     4.289587   
3      NaN                         NaN   Männlich     NaN     7.808564   
4      NaN                         NaN   Männlich  Berlin     4.827586   
..     ...                         ...        ...     ...          ...   
60     Ü65                         NaN   Männlich     NaN     6.226766   
61     Ü65                         NaN   Weiblich     NaN     1.906275   
62     Ü65             dicht besiedelt        NaN     NaN     3.819918   
63     Ü65            gering besiedelt        NaN     NaN     5.040323   
64     Ü65  mittlere Besiedlungsdichte        NaN     NaN     3.435805   

                                                                 ...  \
pv01 ANDERE PARTEI    CDU-CSU DIE GRUENEN  DIE LINKE        FDP  ...   
0         3.606557  21.639344   26.885246   9.180328   7.213115  ...   
1         5.641422  23.106646   14.992272  12.905719  12.210201  ...   
2         3.412534  25.912933   29.835752   5.469622  11.880083  ...   
3         3.803526  24.055416   24.382872   7.506297  13.173804  ...   
4         2.068966  17.931034   27.586207   7.586207  12.413793  ...   
..             ...        ...         ...        ...        ...  ...   
60        0.743494  30.762082   14.962825   5.947955  13.661710  ...   
61        1.985703  36.219222   17.553614   6.592534  10.484512  ...   
62        1.227831  30.422920   17.462483   4.911323   8.867667  ...   
63        2.016129  35.483871   13.709677   8.266129  11.693548  ...   
64        1.265823  35.081374   16.817360   6.329114  14.104882  ...   

     absolute_2021                      absolute_2023                        \
pv01           SPD WUERDE NICHT WAEHLEN           AFD ANDERE PARTEI CDU-CSU   
0               60                   27            22             6      45   
1              159                   92           294            49     267   
2              961                  243           480           215    1609   
3              618                  147           526           156     940   
4               27                   13            15             3      20   
..             ...                  ...           ...           ...     ...   
60             267                   31            88             8     385   
61             247                   71            57            23     426   
62             202                   42            32             6     229   
63              96                   22            46            13     193   
64             216                   38            67            12     389   

                                                            
pv01 DIE GRUENEN DIE LINKE  FDP   SPD WUERDE NICHT WAEHLEN  
0             84        78   19    60                   23  
1            146       140   77   252                   70  
2           1518       240  641  1389                  318  
3            760       225  442   816                  214  
4             36        41    7    29                   12  
..           ...       ...  ...   ...                  ...  
60           154        54   92   321                   36  
61           219        41   94   388                   52  
62           101        37   84   292                   34  
63            65        24   22   117                   17  
64           207        34   80   300                   37  

[65 rows x 36 columns]

In [25]:
# to remove the double index ot the columns, we bring them to one name in one line
new_columns=[]
for index in comp_party_pivot:
    if index[1]!="":
        new_columns.append(index[1]+"_"+index[0])
    else:
        new_columns.append(index[0])
new_columns
comp_party_pivot.columns=new_columns
comp_party_pivot

,Alter,Besiedelung,Geschlecht,OstWest,AFD_percent_2021,ANDERE PARTEI_percent_2021,CDU-CSU_percent_2021,DIE GRUENEN_percent_2021,DIE LINKE_percent_2021,FDP_percent_2021,...,SPD_absolute_2021,WUERDE NICHT WAEHLEN_absolute_2021,AFD_absolute_2023,ANDERE PARTEI_absolute_2023,CDU-CSU_absolute_2023,DIE GRUENEN_absolute_2023,DIE LINKE_absolute_2023,FDP_absolute_2023,SPD_absolute_2023,WUERDE NICHT WAEHLEN_absolute_2023
0,NaN,NaN,NaN,Berlin,2.950820,3.606557,21.639344,26.885246,9.180328,7.213115,...,60,27,22,6,45,84,78,19,60,23
1,NaN,NaN,NaN,Ost,11.746522,5.641422,23.106646,14.992272,12.905719,12.210201,...,159,92,294,49,267,146,140,77,252,70
2,NaN,NaN,NaN,West,4.289587,3.412534,25.912933,29.835752,5.469622,11.880083,...,961,243,480,215,1609,1518,240,641,1389,318
3,NaN,NaN,Männlich,NaN,7.808564,3.803526,24.055416,24.382872,7.506297,13.173804,...,618,147,526,156,940,760,225,442,816,214
4,NaN,NaN,Männlich,Berlin,4.827586,2.068966,17.931034,27.586207,7.586207,12.413793,...,27,13,15,3,20,36,41,7,29,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Ü65,NaN,Männlich,NaN,6.226766,0.743494,30.762082,14.962825,5.947955,13.661710,...,267,31,88,8,385,154,54,92,321,36
61,Ü65,NaN,Weiblich,NaN,1.906275,1.985703,36.219222,17.553614,6.592534,10.484512,...,247,71,57,23,426,219,41,94,388,52
62,Ü65,dicht besiedelt,NaN,NaN,3.819918,1.227831,30.422920,17.462483,4.911323,8.867667,...,202,42,32,6,229,101,37,84,292,34
63,Ü65,gering besiedelt,NaN,NaN,5.040323,2.016129,35.483871,13.709677,8.266129,11.693548,...,96,22,46,13,193,65,24,22,117,17


In [26]:
comp_factor

,Alter,Besiedelung,Geschlecht,OstWest,absolute_2021,percent_2021,absolute_2023,percent_2023
8,NaN,NaN,NaN,West,1350,26.769780,1403,25.804672
14,NaN,NaN,Weiblich,NaN,990,29.649596,1032,29.037704
18,Ü65,NaN,NaN,NaN,883,46.449237,853,41.793239
19,NaN,mittlere Besiedlungsdichte,NaN,NaN,845,30.007102,921,29.623673
20,NaN,NaN,Männlich,NaN,841,27.673577,916,27.631976
...,...,...,...,...,...,...,...,...
162,U30,dicht besiedelt,NaN,NaN,19,4.460094,28,6.436782
164,30-49,NaN,NaN,Berlin,18,18.556701,7,7.865169
191,U30,NaN,NaN,Berlin,0,0.000000,1,2.222222
192,NaN,mittlere Besiedlungsdichte,NaN,Berlin,0,NaN,0,NaN


In [27]:
#and then again we merge both datasets
comp_migration_party_pivot=pd.merge(comp_factor,comp_party_pivot,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest'])
comp_migration_party_pivot

,Alter,Besiedelung,Geschlecht,OstWest,absolute_2021,percent_2021,absolute_2023,percent_2023,AFD_percent_2021,ANDERE PARTEI_percent_2021,...,SPD_absolute_2021,WUERDE NICHT WAEHLEN_absolute_2021,AFD_absolute_2023,ANDERE PARTEI_absolute_2023,CDU-CSU_absolute_2023,DIE GRUENEN_absolute_2023,DIE LINKE_absolute_2023,FDP_absolute_2023,SPD_absolute_2023,WUERDE NICHT WAEHLEN_absolute_2023
0,NaN,NaN,NaN,West,1350,26.769780,1403,25.804672,4.289587,3.412534,...,961,243,480,215,1609,1518,240,641,1389,318
1,NaN,NaN,Weiblich,NaN,990,29.649596,1032,29.037704,3.076923,3.769231,...,562,215,270,114,981,988,233,295,885,197
2,Ü65,NaN,NaN,NaN,883,46.449237,853,41.793239,3.897216,1.413276,...,514,102,145,31,811,373,95,186,709,88
3,NaN,mittlere Besiedlungsdichte,NaN,NaN,845,30.007102,921,29.623673,5.478662,3.835063,...,537,133,369,143,913,711,158,381,721,173
4,NaN,NaN,Männlich,NaN,841,27.673577,916,27.631976,7.808564,3.803526,...,618,147,526,156,940,760,225,442,816,214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,U30,dicht besiedelt,NaN,NaN,19,4.460094,28,6.436782,2.307692,3.653846,...,41,22,21,14,46,183,67,39,96,22
61,30-49,NaN,NaN,Berlin,18,18.556701,7,7.865169,3.809524,10.476190,...,10,6,14,2,7,39,24,4,15,8
62,U30,NaN,NaN,Berlin,0,0.000000,1,2.222222,0.000000,0.000000,...,10,9,3,1,0,15,24,0,14,0
63,NaN,mittlere Besiedlungsdichte,NaN,Berlin,0,NaN,0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [28]:
comp_migration_party_pivot.columns

Index(['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', 'absolute_2021',
       'percent_2021', 'absolute_2023', 'percent_2023', 'AFD_percent_2021',
       'ANDERE PARTEI_percent_2021', 'CDU-CSU_percent_2021',
       'DIE GRUENEN_percent_2021', 'DIE LINKE_percent_2021',
       'FDP_percent_2021', 'SPD_percent_2021',
       'WUERDE NICHT WAEHLEN_percent_2021', 'AFD_percent_2023',
       'ANDERE PARTEI_percent_2023', 'CDU-CSU_percent_2023',
       'DIE GRUENEN_percent_2023', 'DIE LINKE_percent_2023',
       'FDP_percent_2023', 'SPD_percent_2023',
       'WUERDE NICHT WAEHLEN_percent_2023', 'AFD_absolute_2021',
       'ANDERE PARTEI_absolute_2021', 'CDU-CSU_absolute_2021',
       'DIE GRUENEN_absolute_2021', 'DIE LINKE_absolute_2021',
       'FDP_absolute_2021', 'SPD_absolute_2021',
       'WUERDE NICHT WAEHLEN_absolute_2021', 'AFD_absolute_2023',
       'ANDERE PARTEI_absolute_2023', 'CDU-CSU_absolute_2023',
       'DIE GRUENEN_absolute_2023', 'DIE LINKE_absolute_2023',
       'FDP_absol

In [29]:
#we take in the data from the zensus that gives a share of the population for each subgroup
#but only with two features defined as the same time
zensus_two_level=pd.read_csv("json_none.csv")
zensus_two_level

,Geschlecht,Besiedelung,OstWest,Alter,Share
0,NaN,NaN,Ost,U30,1.675804
1,NaN,gering besiedelt,NaN,U30,2.543912
2,NaN,dicht besiedelt,Ost,NaN,3.853220
3,NaN,NaN,Ost,30-49,4.559863
4,NaN,NaN,Ost,50-64,4.751534
5,NaN,NaN,Ost,Ü65,5.482466
6,NaN,gering besiedelt,Ost,NaN,5.491600
7,NaN,gering besiedelt,NaN,30-49,5.723338
8,NaN,mittlere Besiedlungsdichte,NaN,U30,5.842770
9,NaN,dicht besiedelt,NaN,U30,6.005464


In [30]:
# we merge this
comp_migration_party_pivot_zensus=pd.merge(comp_migration_party_pivot,zensus_two_level,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest'])


In [31]:
#and we filter only for the percentage results  
perc_share_df=comp_migration_party_pivot_zensus[comp_migration_party_pivot_zensus.columns[~comp_migration_party_pivot_zensus.columns.str.contains("absolute")]]

In [32]:
perc_share_df

,Alter,Besiedelung,Geschlecht,OstWest,percent_2021,percent_2023,AFD_percent_2021,ANDERE PARTEI_percent_2021,CDU-CSU_percent_2021,DIE GRUENEN_percent_2021,...,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023,Share
0,NaN,NaN,NaN,West,26.769780,25.804672,4.289587,3.412534,25.912933,29.835752,...,3.874980,7.488300,3.354134,25.101404,23.681747,3.744150,10.000000,21.669267,4.960998,79.480028
1,NaN,NaN,Weiblich,NaN,29.649596,29.037704,3.076923,3.769231,26.538462,30.230769,...,5.512821,6.813020,2.876609,24.753974,24.930608,5.879384,7.443856,22.331567,4.970982,51.683982
2,Ü65,NaN,NaN,NaN,46.449237,41.793239,3.897216,1.413276,33.704497,16.359743,...,4.368308,5.947498,1.271534,33.264971,15.299426,3.896637,7.629204,29.081214,3.609516,29.096249
3,NaN,mittlere Besiedlungsdichte,NaN,NaN,30.007102,29.623673,5.478662,3.835063,26.614764,25.403691,...,3.835063,10.339031,4.006725,25.581395,19.921547,4.427010,10.675259,20.201737,4.847296,44.144116
4,NaN,NaN,Männlich,NaN,27.673577,27.631976,7.808564,3.803526,24.055416,24.382872,...,3.702771,12.895317,3.824467,23.044864,18.632018,5.516058,10.835989,20.004903,5.246384,48.259419
5,NaN,NaN,Weiblich,West,27.685950,27.333805,2.173913,3.228900,26.854220,33.024297,...,5.051151,4.979515,2.741885,25.969114,27.103687,3.844942,7.910495,22.313268,5.137094,41.086342
6,Ü65,NaN,NaN,West,44.265594,38.835573,2.500000,1.277778,34.500000,18.944444,...,3.555556,3.771608,1.047669,35.463594,17.653222,1.257203,8.486118,28.810896,3.509691,22.535343
7,NaN,mittlere Besiedlungsdichte,NaN,West,27.928693,27.052023,4.245122,3.183841,27.114002,27.559055,...,3.457720,8.461538,3.846154,26.120401,21.839465,3.311037,11.605351,20.033445,4.782609,37.019270
8,NaN,NaN,Männlich,West,25.745485,24.147183,6.395164,3.595291,24.976137,26.662424,...,2.704423,9.947482,3.954279,24.250850,20.327464,3.645351,12.048193,21.037998,4.788384,38.393686
9,Ü65,NaN,Weiblich,NaN,46.785714,45.984113,1.906275,1.985703,36.219222,17.553614,...,5.639396,4.384615,1.769231,32.769231,16.846154,3.153846,7.230769,29.846154,4.000000,16.354651


In [33]:
perc_share_df=perc_share_df.round(1)
perc_share_df


,Alter,Besiedelung,Geschlecht,OstWest,percent_2021,percent_2023,AFD_percent_2021,ANDERE PARTEI_percent_2021,CDU-CSU_percent_2021,DIE GRUENEN_percent_2021,...,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023,Share
0,NaN,NaN,NaN,West,26.8,25.8,4.3,3.4,25.9,29.8,...,3.9,7.5,3.4,25.1,23.7,3.7,10.0,21.7,5.0,79.5
1,NaN,NaN,Weiblich,NaN,29.6,29.0,3.1,3.8,26.5,30.2,...,5.5,6.8,2.9,24.8,24.9,5.9,7.4,22.3,5.0,51.7
2,Ü65,NaN,NaN,NaN,46.4,41.8,3.9,1.4,33.7,16.4,...,4.4,5.9,1.3,33.3,15.3,3.9,7.6,29.1,3.6,29.1
3,NaN,mittlere Besiedlungsdichte,NaN,NaN,30.0,29.6,5.5,3.8,26.6,25.4,...,3.8,10.3,4.0,25.6,19.9,4.4,10.7,20.2,4.8,44.1
4,NaN,NaN,Männlich,NaN,27.7,27.6,7.8,3.8,24.1,24.4,...,3.7,12.9,3.8,23.0,18.6,5.5,10.8,20.0,5.2,48.3
5,NaN,NaN,Weiblich,West,27.7,27.3,2.2,3.2,26.9,33.0,...,5.1,5.0,2.7,26.0,27.1,3.8,7.9,22.3,5.1,41.1
6,Ü65,NaN,NaN,West,44.3,38.8,2.5,1.3,34.5,18.9,...,3.6,3.8,1.0,35.5,17.7,1.3,8.5,28.8,3.5,22.5
7,NaN,mittlere Besiedlungsdichte,NaN,West,27.9,27.1,4.2,3.2,27.1,27.6,...,3.5,8.5,3.8,26.1,21.8,3.3,11.6,20.0,4.8,37.0
8,NaN,NaN,Männlich,West,25.7,24.1,6.4,3.6,25.0,26.7,...,2.7,9.9,4.0,24.3,20.3,3.6,12.0,21.0,4.8,38.4
9,Ü65,NaN,Weiblich,NaN,46.8,46.0,1.9,2.0,36.2,17.6,...,5.6,4.4,1.8,32.8,16.8,3.2,7.2,29.8,4.0,16.4


In [34]:
perc_share_df["Geschlecht"]=perc_share_df["Geschlecht"].str.replace("Männlich","Maennlich")
perc_share_df["Alter"]=perc_share_df["Alter"].str.replace("Ü65","65plus")
perc_share_df["Besiedelung"]=perc_share_df["Besiedelung"].str.replace("dicht besiedelt","Stadt")
perc_share_df["Besiedelung"]=perc_share_df["Besiedelung"].str.replace("gering besiedelt","Dorf")
perc_share_df["Besiedelung"]=perc_share_df["Besiedelung"].str.replace("mittlere Besiedlungsdichte","Kleinstadt")
perc_share_df

,Alter,Besiedelung,Geschlecht,OstWest,percent_2021,percent_2023,AFD_percent_2021,ANDERE PARTEI_percent_2021,CDU-CSU_percent_2021,DIE GRUENEN_percent_2021,...,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023,Share
0,NaN,NaN,NaN,West,26.8,25.8,4.3,3.4,25.9,29.8,...,3.9,7.5,3.4,25.1,23.7,3.7,10.0,21.7,5.0,79.5
1,NaN,NaN,Weiblich,NaN,29.6,29.0,3.1,3.8,26.5,30.2,...,5.5,6.8,2.9,24.8,24.9,5.9,7.4,22.3,5.0,51.7
2,65plus,NaN,NaN,NaN,46.4,41.8,3.9,1.4,33.7,16.4,...,4.4,5.9,1.3,33.3,15.3,3.9,7.6,29.1,3.6,29.1
3,NaN,Kleinstadt,NaN,NaN,30.0,29.6,5.5,3.8,26.6,25.4,...,3.8,10.3,4.0,25.6,19.9,4.4,10.7,20.2,4.8,44.1
4,NaN,NaN,Maennlich,NaN,27.7,27.6,7.8,3.8,24.1,24.4,...,3.7,12.9,3.8,23.0,18.6,5.5,10.8,20.0,5.2,48.3
5,NaN,NaN,Weiblich,West,27.7,27.3,2.2,3.2,26.9,33.0,...,5.1,5.0,2.7,26.0,27.1,3.8,7.9,22.3,5.1,41.1
6,65plus,NaN,NaN,West,44.3,38.8,2.5,1.3,34.5,18.9,...,3.6,3.8,1.0,35.5,17.7,1.3,8.5,28.8,3.5,22.5
7,NaN,Kleinstadt,NaN,West,27.9,27.1,4.2,3.2,27.1,27.6,...,3.5,8.5,3.8,26.1,21.8,3.3,11.6,20.0,4.8,37.0
8,NaN,NaN,Maennlich,West,25.7,24.1,6.4,3.6,25.0,26.7,...,2.7,9.9,4.0,24.3,20.3,3.6,12.0,21.0,4.8,38.4
9,65plus,NaN,Weiblich,NaN,46.8,46.0,1.9,2.0,36.2,17.6,...,5.6,4.4,1.8,32.8,16.8,3.2,7.2,29.8,4.0,16.4


In [35]:
#and we write this into a csv file
perc_share_df.to_csv("perc_share_df.csv")

In [36]:
perc_share_df.Besiedelung.unique()

array([nan, 'Kleinstadt', 'Stadt', 'Dorf'], dtype=object)

In [37]:
json_array=[]
for index, row in perc_share_df.iterrows():
    print(row)
    base_dict=row[["Alter","Besiedelung","Geschlecht","OstWest","Share"]].to_dict()
    base_dict["migration"]=row[[ 'percent_2021', 'percent_2023']].to_dict()
    base_dict["wahl"]=row[['AFD_percent_2021', 'ANDERE PARTEI_percent_2021',
        'CDU-CSU_percent_2021', 'DIE GRUENEN_percent_2021',
        'DIE LINKE_percent_2021', 'FDP_percent_2021', 'SPD_percent_2021',
        'WUERDE NICHT WAEHLEN_percent_2021', 'AFD_percent_2023',
        'ANDERE PARTEI_percent_2023', 'CDU-CSU_percent_2023',
        'DIE GRUENEN_percent_2023', 'DIE LINKE_percent_2023',
        'FDP_percent_2023', 'SPD_percent_2023',
        'WUERDE NICHT WAEHLEN_percent_2023']].to_dict()
    json_array.append(base_dict)
json_array[0]

Alter                                 NaN
Besiedelung                           NaN
Geschlecht                            NaN
OstWest                              West
percent_2021                         26.8
percent_2023                         25.8
AFD_percent_2021                      4.3
ANDERE PARTEI_percent_2021            3.4
CDU-CSU_percent_2021                 25.9
DIE GRUENEN_percent_2021             29.8
DIE LINKE_percent_2021                5.5
FDP_percent_2021                     11.9
SPD_percent_2021                     15.3
WUERDE NICHT WAEHLEN_percent_2021     3.9
AFD_percent_2023                      7.5
ANDERE PARTEI_percent_2023            3.4
CDU-CSU_percent_2023                 25.1
DIE GRUENEN_percent_2023             23.7
DIE LINKE_percent_2023                3.7
FDP_percent_2023                     10.0
SPD_percent_2023                     21.7
WUERDE NICHT WAEHLEN_percent_2023     5.0
Share                                79.5
Name: 0, dtype: object
Alter      

{'Alter': nan,
 'Besiedelung': nan,
 'Geschlecht': nan,
 'OstWest': 'West',
 'Share': 79.5,
 'migration': {'percent_2021': 26.8, 'percent_2023': 25.8},
 'wahl': {'AFD_percent_2021': 4.3,
  'ANDERE PARTEI_percent_2021': 3.4,
  'CDU-CSU_percent_2021': 25.9,
  'DIE GRUENEN_percent_2021': 29.8,
  'DIE LINKE_percent_2021': 5.5,
  'FDP_percent_2021': 11.9,
  'SPD_percent_2021': 15.3,
  'WUERDE NICHT WAEHLEN_percent_2021': 3.9,
  'AFD_percent_2023': 7.5,
  'ANDERE PARTEI_percent_2023': 3.4,
  'CDU-CSU_percent_2023': 25.1,
  'DIE GRUENEN_percent_2023': 23.7,
  'DIE LINKE_percent_2023': 3.7,
  'FDP_percent_2023': 10.0,
  'SPD_percent_2023': 21.7,
  'WUERDE NICHT WAEHLEN_percent_2023': 5.0}}

In [38]:
! pip3 install simplejson


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [39]:
import simplejson as json
#And then we write out the file as a json file
with open('data_for_viz_migration_2.json', 'w', encoding='utf-8') as file:
    json.dump(json_array, file, ignore_nan=True, indent=2)


In [40]:
comp_no_berlin_party=comp_party[comp_party["OstWest"]!="Berlin"].sort_values(by=["diff_percent"])
comp_no_berlin_party.head(60)

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute_2021,percent_2021,absolute_2023,percent_2023,diff_percent
149,U30,NaN,NaN,Ost,FDP,28,22.580645,5,3.759398,-18.821247
195,30-49,dicht besiedelt,NaN,NaN,DIE GRUENEN,428,49.365629,292,34.152047,-15.213582
35,30-49,NaN,Männlich,NaN,DIE GRUENEN,354,31.355182,209,17.666948,-13.688233
267,50-64,mittlere Besiedlungsdichte,NaN,NaN,DIE GRUENEN,325,32.146390,194,18.671800,-13.474590
211,U30,dicht besiedelt,NaN,NaN,DIE GRUENEN,262,50.384615,183,37.500000,-12.884615
355,NaN,dicht besiedelt,Männlich,NaN,DIE GRUENEN,493,36.437546,337,24.123121,-12.314425
59,50-64,NaN,Weiblich,NaN,DIE GRUENEN,346,34.088670,237,21.964782,-12.123888
475,NaN,dicht besiedelt,NaN,West,DIE GRUENEN,860,39.907193,619,28.123580,-11.783612
241,U30,gering besiedelt,NaN,NaN,ANDERE PARTEI,24,12.371134,3,1.612903,-10.758231
499,NaN,dicht besiedelt,NaN,NaN,DIE GRUENEN,1024,37.101449,760,26.722925,-10.378524


In [41]:
comp_no_berlin_party.tail(60)

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute_2021,percent_2021,absolute_2023,percent_2023,diff_percent
392,NaN,mittlere Besiedlungsdichte,Weiblich,NaN,AFD,43,2.489867,135,7.674815,5.184948
242,U30,gering besiedelt,NaN,NaN,CDU-CSU,28,14.432990,37,19.892473,5.459483
46,30-49,NaN,Weiblich,NaN,SPD,119,10.778986,169,16.312741,5.533756
168,50-64,NaN,NaN,West,AFD,72,4.183614,175,9.754738,5.571124
32,30-49,NaN,Männlich,NaN,AFD,102,9.034544,173,14.623838,5.589294
207,50-64,dicht besiedelt,NaN,NaN,WUERDE NICHT WAEHLEN,28,4.375000,69,10.058309,5.683309
200,50-64,dicht besiedelt,NaN,NaN,AFD,41,6.406250,83,12.099125,5.692875
190,Ü65,NaN,NaN,West,SPD,411,22.833333,550,28.810896,5.977562
256,30-49,mittlere Besiedlungsdichte,NaN,NaN,AFD,68,7.456140,126,13.519313,6.063173
0,30-49,NaN,NaN,NaN,AFD,137,6.135244,271,12.212708,6.077464


In [42]:
comp_factor
comp_factor["diff_percent"]=comp_factor["percent_2023"]-comp_factor["percent_2021"]
comp_no_berlin_migration=comp_factor[comp_factor["OstWest"]!="Berlin"]

comp_no_berlin_migration=comp_no_berlin_migration.sort_values("diff_percent")

In [43]:
comp_no_berlin_migration.head(20)

,Alter,Besiedelung,Geschlecht,OstWest,absolute_2021,percent_2021,absolute_2023,percent_2023,diff_percent
57,Ü65,NaN,Männlich,NaN,359,45.966709,332,36.563877,-9.402833
47,Ü65,mittlere Besiedlungsdichte,NaN,NaN,424,46.644664,372,38.750000,-7.894664
108,30-49,dicht besiedelt,NaN,NaN,99,15.739269,56,8.736349,-7.002919
29,Ü65,NaN,NaN,West,660,44.265594,607,38.835573,-5.430021
80,30-49,NaN,Weiblich,NaN,207,24.013921,178,18.638743,-5.375178
18,Ü65,NaN,NaN,NaN,883,46.449237,853,41.793239,-4.655999
98,50-64,gering besiedelt,NaN,NaN,140,35.000000,131,30.536131,-4.463869
70,NaN,gering besiedelt,Männlich,NaN,261,40.340031,246,35.912409,-4.427622
69,Ü65,dicht besiedelt,NaN,NaN,263,42.215088,250,38.819876,-3.395213
43,NaN,mittlere Besiedlungsdichte,Weiblich,NaN,489,33.062880,481,29.820211,-3.242670


In [44]:
comp_no_berlin_migration.tail(20)

,Alter,Besiedelung,Geschlecht,OstWest,absolute_2021,percent_2021,absolute_2023,percent_2023,diff_percent
162,U30,dicht besiedelt,NaN,NaN,19,4.460094,28,6.436782,1.976688
113,30-49,NaN,NaN,Ost,86,29.152542,97,31.493506,2.340964
111,U30,NaN,NaN,NaN,91,9.660297,127,12.129895,2.469598
121,U30,NaN,NaN,West,71,9.033079,102,11.604096,2.571017
148,U30,mittlere Besiedlungsdichte,NaN,NaN,32,9.495549,52,12.149533,2.653984
58,NaN,mittlere Besiedlungsdichte,Männlich,NaN,356,26.626776,440,29.411765,2.784988
138,U30,gering besiedelt,NaN,NaN,40,22.346369,47,25.543478,3.197110
74,NaN,gering besiedelt,Weiblich,NaN,229,34.539970,280,37.991859,3.451889
79,50-64,NaN,Weiblich,NaN,219,24.579125,271,28.199792,3.620667
139,U30,NaN,Weiblich,NaN,40,8.583691,62,12.277228,3.693537
